In [ ]:
!pip install deepvision-toolkit

In [ ]:
import deepvision
import torch
import tensorflow as tf

config = {
    'batch_size': 32,
    'epochs': 5
}

# Pre-Train on Imagenette using DeepVision and TensorFlow

To simulate larger-scale pre-training on a dataset such as Imagenet, or a proprietary dataset.

In [ ]:
import tensorflow_datasets as tfds

(train_set, test_set), info = tfds.load("imagenette", 
                                           split=["train", "validation"],
                                           as_supervised=True, with_info=True)

class_names = info.features["label"].names
n_classes = info.features["label"].num_classes

def preprocess_img(img, label):
    img = tf.image.resize(img, (224, 224))
    img = img/255
    return img, label

train_set = train_set.map(preprocess_img).batch(config['batch_size']).prefetch(tf.data.AUTOTUNE)
test_set = test_set.map(preprocess_img).batch(config['batch_size'], drop_remainder=True).prefetch(tf.data.AUTOTUNE)

In [ ]:
tf_model = deepvision.models.EfficientNetV2B0(include_top=True,
                                       classes=10,
                                       input_shape=(224, 224, 3),
                                       backend='tensorflow')

tf_model.compile(
  loss=tf.keras.losses.SparseCategoricalCrossentropy(),
  optimizer=tf.keras.optimizers.Adam(learning_rate=1e-5),
  metrics=['accuracy']
)
history = tf_model.fit(train_set, epochs=5, validation_data=test_set)

In [ ]:
tf_model.save('effnet.h5')

# Transfer TensorFlow Weights to PyTorch

Using DeepVision models, you can easily pick up a trained checkpoint of a model trained in *either* TensorFlow or PyTorch, and fine-tune with either libraries/ecosystems. Each model comes with its `model_weight_mapper` which exposes a `load()` function, accepting a `filepath` to the trained model, the `origin` library and `target` library.

You'll need to supply a dummy input in the *origin library's* expected format, and DeepVision will reconstruct the target library's equivalent of the model, with the loaded and converted weights, ready to fine-tune:

In [ ]:
from deepvision.models.classification.efficientnet import efficientnet_weight_mapper

In [ ]:
dummy_input = tf.random.normal([1, 224, 224, 3])
pt_model = efficientnet_weight_mapper.load(filepath='effnet.h5', 
                                           origin='tensorflow', 
                                           target='pytorch', 
                                           dummy_input=dummy_input)

# If the top classes differ, you can set them here
pt_model.top_dense = torch.nn.Linear(pt_model.top_dense.in_features, 10)

# Fine-Tune in PyTorch

In [ ]:
from torchvision import transforms
from torchvision.datasets import CIFAR10
from torch.utils.data import DataLoader
import pytorch_lightning as pl

device = 'cuda' if torch.cuda.is_available() else 'cpu'

transform=transforms.Compose([transforms.ToTensor(),
                              transforms.Resize([224, 224])])

cifar_train = CIFAR10('cifar10', train=True, download=True, transform=transform)
cifar_test = CIFAR10('cifar10', train=False, download=True, transform=transform)

train_dataloader = DataLoader(cifar_train, batch_size=config['batch_size'], drop_last=True)
val_dataloader = DataLoader(cifar_test, batch_size=config['batch_size'], drop_last=True)

In [ ]:
loss = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(pt_model.parameters(), 1e-6)

pt_model.compile(loss=loss, optimizer=optimizer)
trainer = pl.Trainer(accelerator=device, max_epochs=5)
trainer.fit(pt_model, train_dataloader, val_dataloader)

# Pre-Train using PyTorch

In [ ]:
import torch

from torchvision import transforms
from torchvision.datasets import CIFAR10
from torch.utils.data import DataLoader
import pytorch_lightning as pl
from pytorch_lightning.callbacks import RichProgressBar

device = 'cuda' if torch.cuda.is_available() else 'cpu'

transform=transforms.Compose([transforms.ToTensor(),
                              transforms.Resize([224, 224])])

cifar_train = CIFAR10('cifar10', train=True, download=True, transform=transform)
cifar_test = CIFAR10('cifar10', train=False, download=True, transform=transform)

train_dataloader = DataLoader(cifar_train, batch_size=config['batch_size'], drop_last=True)
val_dataloader = DataLoader(cifar_test, batch_size=config['batch_size'], drop_last=True)

pt_model = deepvision.models.EfficientNetV2B0(include_top=True,
                                       classes=10,
                                       input_shape=(3, 224, 224),
                                       backend='pytorch')

loss = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(pt_model.parameters(), 1e-5)

pt_model.compile(loss=loss, optimizer=optimizer)
trainer = pl.Trainer(accelerator=device, max_epochs=config['epochs'])
trainer.fit(pt_model, train_dataloader, val_dataloader)

In [ ]:
torch.save(pt_model.state_dict(), 'effnet.pt')

# Transfer PyTorch Weights to TensorFlow

In [ ]:
from deepvision.models.classification.efficientnet import efficientnet_weight_mapper

dummy_input_torch = torch.ones(1, 3, 224, 224)
kwargs = {'include_top': True, 'classes': 10, 'input_shape':(3, 224, 224)}
tf_model = efficientnet_weight_mapper.load_pt_to_tf(filepath='effnet.pt',
                                architecture='EfficientNetV2B0',
                                kwargs=kwargs,
                                dummy_input=dummy_input_torch)

# Fine-Tune in TensorFlow

In [ ]:
import tensorflow_datasets as tfds
import tensorflow as tf

(train_set, test_set), info = tfds.load("imagenette", 
                                           split=["train", "validation"],
                                           as_supervised=True, with_info=True)

class_names = info.features["label"].names
n_classes = info.features["label"].num_classes

def preprocess_img(img, label):
    img = tf.image.resize(img, (224, 224))
    img = img/255
    return img, label

train_set = train_set.map(preprocess_img).batch(config['batch_size']).prefetch(tf.data.AUTOTUNE)
test_set = test_set.map(preprocess_img).batch(config['batch_size'], drop_remainder=True).prefetch(tf.data.AUTOTUNE)

tf_model.compile(
  loss=tf.keras.losses.SparseCategoricalCrossentropy(),
  optimizer=tf.keras.optimizers.Adam(learning_rate=1e-5),
  metrics=['accuracy']
)
history = tf_model.fit(train_set, epochs=5, validation_data=test_set)